In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns  # This optional package makes plots prettier

import openmc

## The statepoint file

Let's begin by loading the statepoint HDF5 file for batch 50 into the Python API's `openmc.StatePoint` class:

In [ ]:
sp = openmc.StatePoint('statepoint.50.h5')

The statepoint file contains some metadata like the date and time that it was written.

In [ ]:
sp.date_and_time

OpenMC computes a number of estimators for the multiplication factor $k_\text{eff}$. Each of these estimators is stored as an attribute in the `StatePoint` class. We can report an ensemble-averaged estimator through the `StatePoint.k_combined` property with an `ndarray` storing the mean and uncertainty.

In [ ]:
sp.k_combined

## Convergence of k and entropy

The statepoint file records the $k_\text{eff}$ and entropy values for every neutron generation.  These values should be converged in the inactive batches.  Let's plot them with Matplotlib.

In [ ]:
plt.plot(sp.k_generation)

In [ ]:
plt.plot(sp.entropy)

It looks like entropy still had some long-term trends in the active batches between about 10 and 40.  That means are tallies probably have some bias in them.  A proper calculation should use more inactive batches (or a better initial guess on the source distribution), but this is good enough for demonstration purposes.

## Getting Tally objects from the statepoint

Our `StatePoint` also has a number of different `Tally` objects stored in the `StatePoint.tallies` as a `dict`. We can inspect one the `Tally` objects by directly indexing into `tallies` with the appropriate ID:

In [ ]:
sp.tallies.keys()

In [ ]:
sp.tallies[10000]

Sometimes you may not know the ID number of the tally.  So in order to find the tally you need, there is a `get_tally(...)` method which allows one to retrieve a `Tally` object with an arbitrary subset or combination of the following information:

* integer ID
* string name
* one or more string scores
* one or more `Filter` objects
* one or more string nuclides

As an example, let's retrieve the `Tally` using its string name:

In [ ]:
mesh_fiss = sp.get_tally(name='mesh fission')

As we can see, this has extracted the `Tally` with the name `"mesh_fiss"` into a new Python variable called `mesh_fiss`. Let's extract a few other `Tally` objects into their own respective variables.

In [ ]:
flux = sp.get_tally(name='flux')
distribcell = sp.get_tally(name='distribcell')

## Pandas DataFames for tallies

The easiest way to analyze tallies in the Python API is with the [Pandas](http://pandas.pydata.org/) Python package.

In [ ]:
df = mesh_fiss.get_pandas_dataframe()
df.head(10)  # Show the first 10 rows

In [ ]:
df = flux.get_pandas_dataframe()
df.head(10)

In [ ]:
df = distribcell.get_pandas_dataframe()
df.head(10)

Pandas does more than just format tables.  Pandas is a very powerful data processing tool.  One of the most important features is "fancy indexing".  This allows you to quickly isolate the data you want using a Boolean expression.  For example, here's how to focus on reaction rates for U-235.

In [ ]:
df = mesh_fiss.get_pandas_dataframe()
indices = df['nuclide'] == 'U235'
indices.head(5)

In [ ]:
sub_df = df[df['nuclide'] == 'U235']
sub_df.head(5)

Now consider the case where we wish to select from `sub_df` those mesh cells which have a "fission" rate that is above the average:

In [ ]:
# Replace 0's with NaN to eliminate them from average.
sub_df = sub_df.replace(0, np.nan)

# Extract rows corresponding to above-average fission rates.
indices = sub_df['mean'] > sub_df['mean'].mean()
above_avg = sub_df[indices]
above_avg.head(5)

Finally, let's use Pandas fancy indexing to select the data corresponding to those mesh cells in the lower left triangle of the mesh:

In [ ]:
indices = df[('mesh 10001', 'x')] > df[('mesh 10001', 'y')]
lower = df[indices]
lower.head(5)

## Plotting flux tallies

Next, let's use Matplotlib to plot the flux energy spectrum.  The first step is to get the DataFrame for the flux tally and extract the mean values.

In [ ]:
df = flux.get_pandas_dataframe()
df.head(5)

In [ ]:
# Extract the flux mean values array.
fluxes = df['mean'].values

# Extend the flux array for Matplotlib's step plot.
fluxes = np.insert(fluxes, 0, fluxes[0])

We also need an array of the energy grid points.  We could get that array from the `'energy low [eV]'` and the `'energy high [eV]'` columns of the DataFrame, but it is easier to extract from the `EnergyFilter` attached to the tally.

In [ ]:
# Extract the energy bins from the Tally's EnergyFilter
energy_filter = flux.find_filter(openmc.EnergyFilter)
energies = energy_filter.bins

In [ ]:
fig = plt.figure()
plt.loglog(energies, fluxes, drawstyle='steps', c='r')
plt.xlabel('Energy [eV]')
plt.ylabel('Flux')

## Potting mesh tallies

Mesh tallies can be quickly plotted with the `matplotlib.imshow` function.

In [ ]:
df = mesh_fiss.get_pandas_dataframe()
df.head(5)

In [ ]:
mean = df[df['nuclide'] == 'U235']['mean'].values
rel_err = df[df['nuclide'] == 'U235']['std. dev.'].values / mean

In [ ]:
# Reshape the arrays.
mean.shape = (17, 17)
rel_err.shape = (17, 17)

# Transpose them to match the order expected by imshow.
mean = mean.T
rel_err = rel_err.T

In [ ]:
# Plot the mean on the left.
fig = plt.subplot(121)
plt.imshow(mean, interpolation='none', cmap='jet')
plt.ylim(plt.ylim()[::-1])  # Invert the y-axis.
plt.title('Mean')
plt.grid(False)

# Plot the uncertainty on the right.
fig2 = plt.subplot(122)
plt.imshow(rel_err, interpolation='none', cmap='jet')
plt.ylim(plt.ylim()[::-1])  # Invert the y-axis.
plt.title('Rel. Unc.')
plt.grid(False)

This mesh tally of fission rates particularly informative since the dark blue guide tube do not have any fission and skew the color bar. We can rectify this issue with a little trick to Matplotlib's color scheme as follows.

In [ ]:
# Assign a NaN to zero fission rates in guide tubes
# Matplotlib will ignore "bad" values in the colorbar
mean[mean == 0.] = np.nan
cmap = plt.get_cmap('jet')
cmap.set_bad(alpha=0.)

# Plot the mean on the left.
fig = plt.subplot(121)
plt.imshow(mean, interpolation='none', cmap='jet')
plt.ylim(plt.ylim()[::-1])  # Invert the y-axis.
plt.title('Mean')
plt.grid(False)

# Plot the uncertainty on the right.
fig2 = plt.subplot(122)
plt.imshow(rel_err, interpolation='none', cmap='jet')
plt.ylim(plt.ylim()[::-1])  # Invert the y-axis.
plt.title('Rel. Unc.')
plt.grid(False)

## Other ways to get tally data

Note that Pandas DataFrames are not the only way to interact with tally data!  If DataFrames do not work with your use case, try the `Tally.get_value()` method or accessing `Tally.mean` and `Tally.std_dev` directly.